### Merge experimental results

In [1]:
import pandas as pd
import os
import json

import numpy as np

In [2]:
result_dir = '../logs/json/'

dir_names = sorted([name for name in os.listdir(result_dir) \
    if 'bf20' in name])
file_paths = sorted([os.path.join(result_dir, name, 'season_1.json') for name in dir_names])

print(len(file_paths))

4


In [3]:
""" all season test together
"""

from decimal import Decimal

def decimal_float(num):
    """ round-up
    """
    return Decimal(str(num)).quantize(Decimal("0.0001"), rounding = "ROUND_HALF_UP")

res = []
for dir_name in dir_names:
    data_name = dir_name
    try:
        cur_lr_list = pd.read_json(os.path.join(result_dir, dir_name, 'season_1.json'))['lr'].values

        season_1_df = pd.read_json(os.path.join(result_dir, dir_name, 'season_1.json'))
        season_2_df = pd.read_json(os.path.join(result_dir, dir_name, 'season_2.json'))
        season_3_df = pd.read_json(os.path.join(result_dir, dir_name, 'season_3.json'))
        season_4_df = pd.read_json(os.path.join(result_dir, dir_name, 'season_4.json'))
        season_df_list = [season_1_df, season_2_df, season_3_df, season_4_df]
    except:
        continue

    for lr in cur_lr_list:
        cur_lr_res = dict()
        cur_lr_res['data_name'] = data_name
        cur_lr_res['lr'] = float(str(lr).replace('000000000000001', '').replace('4.9999999999999996', '5'))
        cur_lr_season_1 = season_1_df[season_1_df['lr'] == lr].iloc[0]
        cur_lr_season_2 = season_2_df[season_2_df['lr'] == lr].iloc[0]
        cur_lr_season_3 = season_3_df[season_3_df['lr'] == lr].iloc[0]
        cur_lr_season_4 = season_4_df[season_4_df['lr'] == lr].iloc[0]

        cur_lr_res['mean_macF1'] = decimal_float((cur_lr_season_1['metric']['metric'] + cur_lr_season_2['metric']['metric'] \
            + cur_lr_season_3['metric']['metric'] + cur_lr_season_4['metric']['metric'])/4)

        cur_lr_res['season_1_macF1'] = decimal_float(cur_lr_season_1['metric']['metric'])
        cur_lr_res['season_2_macF1'] = decimal_float(cur_lr_season_2['metric']['metric'])
        cur_lr_res['season_3_macF1'] = decimal_float(cur_lr_season_3['metric']['metric'])
        cur_lr_res['season_4_macF1'] = decimal_float(cur_lr_season_4['metric']['metric'])

        res.append(cur_lr_res)

csv_path = './seasonal_res_compare.csv'
pd.DataFrame(res).to_csv(csv_path ,sep='\t', index=False)